In [1]:
import pandas as pd
import pickle
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA


# Univariate AR

In [2]:

# split dataset into train/test sets
def split_dataset(data, size, train_size):
    train_end = int(size * train_size)
    train, test = data[0:train_end], data[train_end:size]
    train = array(split(train, len(train)/4))
    test = array(split(test, len(test)/4))
    return train, test

def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE for each timestep
    for i in range(actual.shape[1]):
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        rmse = sqrt(mse)
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))

def evaluate_model(model_func, train, test):
    history = [x for x in train]
    # walk-forward validation over with the required timesteps
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_sequence = model_func(history)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next timesteps
        history.append(test[i, :])
    predictions = array(predictions)
    # evaluate predictions
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores, predictions, test[:, :, 0]

# convert multivariate data into a series of CO2
def to_series(data):
    series = [dp[:, 0] for dp in data]
    series = array(series).flatten()
    return series

# arima forecast
def arima_forecast(history):
    # convert history into a univariate series
    series = to_series(history)
    # define and fit model
    model = ARIMA(series, order=(4,0,0))
    model_fit = model.fit(disp=False)
    # make forecast
    yhat = model_fit.predict(len(series), len(series)+3)
    return yhat



In [3]:
def univariate_ARIMA(room_data, size, train_size):
    dataset = room_data
    train, test = split_dataset(dataset.values, size, train_size)

    for name, func in models.items():
        score,scores, predictions, test_set = evaluate_model(func, train, test)
        summarize_scores('ARIMA', score,scores)
    return predictions, test_set


## Test on Room 003

In [4]:
models = dict()
models['arima'] = arima_forecast
df = pd.read_csv('data/Run example/MDB 003.csv')
predictions, test_set = univariate_ARIMA(df, 5000,.8)


ARIMA: [11.616] 6.5, 9.1, 12.7, 15.9


# Multivariate

In [5]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
import numpy as np

def split_dataset(data, size, train_size):
    train_end = int(size * train_size)
    train, test = data[0:train_end], data[train_end:size]
    train = array(split(train, len(train)/4))
    test = array(split(test, len(test)/4))
    return train, test

def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score 
    for i in range(actual.shape[1]):
        # calculate rmse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        rmse = sqrt(mse)
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# evaluate a single model
def evaluate_model(model_func, train, test, full_exog):
    history = [x for x in train]
    # walk-forward validation 
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_sequence = model_func(history, full_exog)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history
        history.append(test[i, :])
    predictions = array(predictions)
    # evaluate predictions 
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores, predictions, test[:, :, 0]

# convert windows of multivariate data into a series
def to_series(data):
    series = [dp[:, 0] for dp in data]
    series = array(series).flatten()
    
    
    df_exog = pd.DataFrame()
    for i in range(len(data[0][0])):
        if (i == 0):continue
        exog = [dp[:, i] for dp in data]
        exog = array(exog).flatten()
        df_exog['exog{}'.format(i)] = exog
    return series, df_exog

# arima forecast
def arima_forecast(history, full_exog):
    # convert history into a univariate series and multivariate variables
    series, df_exog = to_series(history)
    exog = df_exog.to_numpy()
    # define the model
    model = ARIMA(series, order=(4,0,0), exog=exog)
    # fit the model
    model_fit = model.fit(disp=False)
    # make forecast
    yhat = model_fit.predict(len(series), len(series)+3, exog=full_exog[len(series):len(series)+4])
    return yhat

models = dict()
models['arima'] = arima_forecast


In [6]:
def multivariate_ARIMA(room_data, size, train_size):
    models['arima'] = arima_forecast
    dataset = room_data
    train, test = split_dataset(dataset.values, size, train_size)
    
    joined_sets = np.concatenate([train,test])
    
    full_exog = pd.DataFrame()
    for i in range(len(joined_sets[0][0])):
        if (i == 0):continue
        exog = [dp[:, i] for dp in joined_sets]
        exog = array(exog).flatten()
        full_exog['exog{}'.format(i)] = exog
    
    full_exog = full_exog.to_numpy()
    for name, func in models.items():

        score,scores, predictions, test_set = evaluate_model(func, train, test, full_exog)
        summarize_scores('ARIMA', score,scores)
    return predictions, test_set


In [7]:
df = pd.read_csv('data/Run example/MDB 003.csv')
predictions, test_set = multivariate_ARIMA(df, 5000,.8)


ARIMA: [11.570] 6.5, 9.2, 12.7, 15.8
